In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
def resize_images(image1_path, image2_path):
    image_1 = cv2.imread(image1_path)
    image_2 = cv2.imread(image2_path)
    return image_1, cv2.resize(image_2, (image_1.shape[1], image_1.shape[0]))

In [3]:
def hide_image(original_image, hidden_image, bytes_to_use):
    if bytes_to_use == 2:
        hidden_mask = 0b11000000
        original_mask = 0b11111100
    elif bytes_to_use == 3:
        hidden_mask = 0b11100000
        original_mask = 0b11111000
    elif bytes_to_use == 4:
        hidden_mask = 0b11110000
        original_mask = 0b11110000
    else:
        return
    
    shift_bytes = 8 - bytes_to_use

    new_image = np.zeros_like(original_image)
    for i in range(original_image.shape[0]):
            for j in range(original_image.shape[1]):
                for k in range(3):
                    original_pixel_nulled = (original_image[i][j][k] & original_mask)
                    hidden_pixel_shifted = (hidden_image[i][j][k] & hidden_mask) >> shift_bytes
                    new_image[i][j][k] = original_pixel_nulled | hidden_pixel_shifted
    
    return new_image

def extract_image(image, bytes_to_use):
    if bytes_to_use == 2:
        mask = 0b00000011
    elif bytes_to_use == 3:
        mask = 0b00000111
    elif bytes_to_use == 4:
        mask = 0b00001111
    shift_bytes = 8 - bytes_to_use
    extracted_image = np.zeros_like(image)
    for i in range(extracted_image.shape[0]):
            for j in range(extracted_image.shape[1]):
                for k in range(3):  
                    hidden_pixel_shifted = (image[i][j][k] & mask) << shift_bytes
                    extracted_image[i][j][k] = hidden_pixel_shifted
                    
    return extracted_image

In [4]:
# Przykładowe użycie
image1_path = "autumn.tif"
image2_path = "bali.tif"
container, hidden = resize_images(image1_path, image2_path)

container_LSB4 = hide_image(container.copy(), hidden, 4)
extracted_LSB4 = extract_image(container_LSB4, 4)

container_LSB3 = hide_image(container.copy(), hidden, 3)
extracted_LSB3 = extract_image(container_LSB3, 3)

container_LSB2 = hide_image(container.copy(), hidden, 2)
extracted_LSB2 = extract_image(container_LSB2, 2)

In [5]:
cv2.imwrite(r'container_LSB4.jpg', container_LSB4)
cv2.imwrite(r'extracted_LSB4.jpg', extracted_LSB4)
cv2.imwrite(r'container_LSB3.jpg', container_LSB3)
cv2.imwrite(r'extracted_LSB3.jpg', extracted_LSB3)
cv2.imwrite(r'container_LSB2.jpg', container_LSB2)
cv2.imwrite(r'extracted_LSB2.jpg', extracted_LSB2)

True

In [6]:
def calculate_normalized_absolute_error(original_image, changed_image):
    diff_image = np.sqrt((original_image[:,:,0].astype(float) - changed_image[:,:,0].astype(float))**2 + \
                     (original_image[:,:,1].astype(float) - changed_image[:,:,1].astype(float))**2 + \
                     (original_image[:,:,2].astype(float) - changed_image[:,:,2].astype(float))**2)
    M, N, L = original_image.shape

    result = np.sum(diff_image) / (M * N)
    return result

In [7]:
print("Quality for LSB2:", round(calculate_normalized_absolute_error(hidden, extracted_LSB2), 2))
print("Quality for LSB3:", round(calculate_normalized_absolute_error(hidden, extracted_LSB3), 2))
print("Quality for LSB4:", round(calculate_normalized_absolute_error(hidden, extracted_LSB4), 2))

Quality for LSB2: 57.57
Quality for LSB3: 29.32
Quality for LSB4: 13.67
